# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [5]:
# Do all imports and installs here
import pandas as pd
import re
from pyspark.sql import SparkSession
import os
import glob
import configparser
from datetime import datetime, timedelta, date
from pyspark.sql import types as T
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear
from pyspark.sql.types import IntegerType, BooleanType, DateType, LongType, StringType, FloatType

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()


In [7]:
spark

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>



- immigration
- us-cities

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### 1.1 Garther immigration data

In [8]:
# Create function get list file
# TODO

In [ ]:
# Garther data

In [9]:
immigration_df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [112]:
immigration_df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [113]:
immigration_df_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [34]:
immigration_df_spark.select(col("i94addr")).show(5)

+-------+
|i94addr|
+-------+
|   null|
|     AL|
|     MI|
|     MA|
|     MA|
+-------+
only showing top 5 rows



In [33]:
immigration_df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [11]:
#write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

#### 1.2 Garther US cities data

In [21]:
us_cities_source_path = 'us-cities-demographics.csv'

In [22]:
us_cities_df_spark = spark.read.csv(us_cities_source_path, header=True, sep=';')

In [23]:
us_cities_df_spark.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [24]:
us_cities_df_spark.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

### 1.3 Garther countries from dictionary

In [25]:
i94_dictionary_source_path ='I94_SAS_Labels_Descriptions.SAS'

In [26]:
i94_dictionary_list = open(i94_dictionary_source_path, 'r').readlines()

In [27]:
i94_country_df_spark = spark.createDataFrame(
    pd.DataFrame(
        [re.search(r"^\s*(\S+)\s*=\s*'\s*(\S.*\S)\s*'.*$", line).groups() for line in i94_dictionary_list[9:298]],
        columns=['country_code', 'country_name']
    )
)

In [30]:
i94_country_df_spark.show(5)

+------------+--------------------+
|country_code|        country_name|
+------------+--------------------+
|         582|MEXICO Air Sea, a...|
|         236|         AFGHANISTAN|
|         101|             ALBANIA|
|         316|             ALGERIA|
|         102|             ANDORRA|
+------------+--------------------+
only showing top 5 rows



### 1.4 Garther airports from dictionary

In [28]:
i94_airport_df_spark = spark.createDataFrame(
    pd.DataFrame(
        [re.search(r"^\s*'\s*(\S+)\s*'\s*=\s*'\s*(\S.*\S)\s*'.*$", line).groups() for line in i94_dictionary_list[302:962]],
        columns=['airport_code', 'airport_name']
    )
)

In [31]:
i94_airport_df_spark.show(5)

+------------+--------------------+
|airport_code|        airport_name|
+------------+--------------------+
|         ALC|           ALCAN, AK|
|         ANC|       ANCHORAGE, AK|
|         BAR|BAKER AAF - BAKER...|
|         DAC|   DALTONS CACHE, AK|
|         PIZ|DEW STATION PT LA...|
+------------+--------------------+
only showing top 5 rows



### 1.5 Garther modes from dictionary

In [30]:
i94_mode_df_spark = spark.createDataFrame(
    pd.DataFrame(
        [re.search(r"^\s*(\S+)\s*=\s*'\s*(\S.*\S)\s*'.*$", line).groups() for line in i94_dictionary_list[972:976]],
        columns=['mode_code', 'mode_name']
    )
)

In [33]:
i94_mode_df_spark.show(5)

+---------+------------+
|mode_code|   mode_name|
+---------+------------+
|        1|         Air|
|        2|         Sea|
|        3|        Land|
|        9|Not reported|
+---------+------------+



### 1.6 Garther states from dictionary

In [ ]:
i94_mode_df_spark = spark.createDataFrame(
    pd.DataFrame(
        [re.search(r"^\s*'(\S+)'\s*=\s*'\s*(\S.*\S)\s*'.*$", line).groups() for line in i94_dictionary_list[981:1036]],
        columns=['state_code', 'state_name']
    )
)

In [35]:
i94_mode_df_spark.show(5)

+----------+----------+
|state_code|state_name|
+----------+----------+
|        AL|   ALABAMA|
|        AK|    ALASKA|
|        AZ|   ARIZONA|
|        AR|  ARKANSAS|
|        CA|CALIFORNIA|
+----------+----------+
only showing top 5 rows



### 1.7 Garther visa from dictionary

In [39]:
i94_visa_df_spark = spark.createDataFrame(
    pd.DataFrame(
        [re.search(r"^\s*(\S+)\s*=\s*(\S.*\S).*$", line).groups() for line in i94_dictionary_list[1046:1049]],
        columns=['visa_code', 'visa_type']
    )
)

In [40]:
i94_visa_df_spark.show(5)

+---------+---------+
|visa_code|visa_type|
+---------+---------+
|        1| Business|
|        2| Pleasure|
|        3|  Student|
+---------+---------+



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here

# Clean immigration
# Clean cities (states)
# Clean i94 airport
# Clean i94 countries



## Standardize for immigration

In [35]:
immigration_df_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [44]:
immigration_df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [203]:
immigration_df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [6]:
@udf(T.TimestampType())
def to_timestamp (d):
    if d:
        return (datetime(1960,1,1) + timedelta(days=int(d)))
    return None


In [10]:
immigration_df_spark = immigration_df_spark \
                            .withColumn("cicid",col("cicid").cast(LongType())) \
                            .withColumn("i94yr",col("i94yr").cast(IntegerType())) \
                            .withColumn("i94mon",col("i94mon").cast(IntegerType())) \
                            .withColumn("i94cit",col("i94cit").cast(IntegerType())) \
                            .withColumn("i94res",col("i94res").cast(IntegerType())) \
                            .withColumn("i94port",col("i94port").cast(IntegerType())) \
                            .withColumn("arrdate",to_timestamp("arrdate")) \
                            .withColumn("i94mode",col("i94mode").cast(IntegerType())) \
                            .withColumn("depdate",to_timestamp("depdate")) \
                            .withColumn("i94bir",col("i94bir").cast(IntegerType())) \
                            .withColumn("i94visa",col("i94visa").cast(IntegerType())) \
                            .withColumn("biryear",col("biryear").cast(IntegerType())) \
                            .withColumn("admnum",col("admnum").cast(LongType())) \


In [11]:
immigration_df_spark.printSchema()

root
 |-- cicid: long (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: integer (nullable = true)
 |-- arrdate: timestamp (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: timestamp (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: long (

In [12]:
immigration_df_spark.show(5)

+-----+-----+------+------+------+-------+-------------------+-------+-------+-------------------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-----------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|            arrdate|i94mode|i94addr|            depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|     admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------------------+-------+-------+-------------------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-----------+-----+--------+
|    6| 2016|     4|   692|   692|   null|2016-04-29 00:00:00|   null|   null|               null|    37|      2|  1.0|    null|    null| null|      T|   null|      U|   null|   1979|10282016|  null|  null|   null| 1897628485| null|      B2|
|    7| 2016|     4|   254|   27

In [18]:
immigration_df_spark.select(col("fltno")).show(5)

+-----+
|fltno|
+-----+
| null|
|00296|
|   93|
|00199|
|00199|
+-----+
only showing top 5 rows



In [25]:
immigration_df_spark_clean = immigration_df_spark.na.fill({'i94addr': 'NA', 'fltno': 'NA'})

In [34]:
immigration_df_spark.count()

3096313

In [32]:
from pyspark.sql.functions import col,isnan, when, count

In [33]:
immigration_df_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in immigration_df_spark.columns]).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|  occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender| insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|    0|       1| 1881250|3088187|    238| 138429|3095921| 138429|    802|    477|414269|2982605|  83627|     0|19549|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+---

## Clean data for immigration

In [35]:
immigration_df_spark_clean = immigration_df_spark.dropna(subset=['visapost'])

In [36]:
immigration_df_spark_clean.count()

1215063

In [20]:
immigration_df_spark.show(5)

+-----+-----+------+------+------+-------+-------------------+-------+-------+-------------------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-----------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|            arrdate|i94mode|i94addr|            depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|     admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------------------+-------+-------+-------------------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-----------+-----+--------+
|    6| 2016|     4|   692|   692|   null|2016-04-29 00:00:00|   null|   null|               null|    37|      2|  1.0|    null|    null| null|      T|   null|      U|   null|   1979|10282016|  null|  null|   null| 1897628485| null|      B2|
|    7| 2016|     4|   254|   27

## Standardize for us-cities

In [25]:
us_cities_df_spark.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

In [9]:
us_cities_df_spark.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [ ]:
# change name column

In [27]:
us_cities_df_spark = us_cities_df_spark\
                        .withColumnRenamed("City", "city") \
                        .withColumnRenamed("State", "state") \
                        .withColumnRenamed("Median Age", "median_age") \
                        .withColumnRenamed("Male Population", "male_population") \
                        .withColumnRenamed("Female Population", "female_population") \
                        .withColumnRenamed("Total Population", "total_population") \
                        .withColumnRenamed("Number of Veterans", "number_of_neterans") \
                        .withColumnRenamed("Foreign-born", "foreign_born") \
                        .withColumnRenamed("Average Household Size", "average_household_size") \
                        .withColumnRenamed("State Code", "state_code") \
                        .withColumnRenamed("Race", "race") \
                        .withColumnRenamed("Count", "count") \

In [28]:
us_cities_df_spark.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            city|        state|median_age|male_population|female_population|total_population|number_of_neterans|foreign_born|average_household_size|state_code|                race|count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Create the immigartion table
2. Create airport
3. Create us-cities


In [ ]:
    immigration_df_spark.createOrReplaceTempView("admissions_table_DF")
    admissions_table = spark.sql("""
        SELECT  DISTINCT admnum   AS admission_nbr,
                         i94res   AS country_code,
                         i94bir   AS age,
                         i94visa  AS visa_code,
                         visatype AS visa_type,
                         gender   AS person_gender
        FROM admissions_table_DF
        ORDER BY country_code
    """)

In [13]:
immigration_df_spark.createOrReplaceTempView("immigration_view")

In [14]:
immigration_fact = spark.sql("""
        SELECT  DISTINCT cicid,
                         i94yr   AS year,
                         i94mon  AS month,
                         i94cit  AS country_code,
                         i94res  AS residence,
                         i94port AS airport_code,
                         arrdate AS arrival_date,
                         i94mode AS mode_code, 
                         i94addr AS state_code,
                         depdate AS departure_date,
                         i94bir  AS age,
                         i94visa AS visa_code,
                         biryear AS birth_year,
                         gender,
                         airline,
                         admnum  AS admission_number,
                         fltno   AS flight_number
        FROM immigration_view
    """)

In [15]:
immigration_fact.printSchema()

root
 |-- cicid: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- country_code: integer (nullable = true)
 |-- residence: integer (nullable = true)
 |-- airport_code: integer (nullable = true)
 |-- arrival_date: timestamp (nullable = true)
 |-- mode_code: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- departure_date: timestamp (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_code: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admission_number: long (nullable = true)
 |-- flight_number: string (nullable = true)



In [17]:
immigration_fact.show(5)

+-----+----+-----+------------+---------+------------+-------------------+---------+----------+-------------------+---+---------+----------+------+-------+----------------+-------------+
|cicid|year|month|country_code|residence|airport_code|       arrival_date|mode_code|state_code|     departure_date|age|visa_code|birth_year|gender|airline|admission_number|flight_number|
+-----+----+-----+------------+---------+------------+-------------------+---------+----------+-------------------+---+---------+----------+------+-------+----------------+-------------+
|  766|2016|    4|         103|      103|        null|2016-04-01 00:00:00|        1|        IL|2016-04-05 00:00:00| 31|        2|      1985|  null|     OS|     55428193433|        00065|
|  867|2016|    4|         104|      104|        null|2016-04-01 00:00:00|        1|        CA|2016-05-04 00:00:00| 50|        2|      1966|     M|     LH|     55437099833|        00408|
|  892|2016|    4|         104|      104|        null|2016-04-01 

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.